###Whole genome vcf files

Input: Individually mapped bam files     
Output: Single VCF file

In [ ]:
import sys
import re
import subprocess
from bwa_mem_pe import *
from consensus_base import *

## structure
consensus_params - running all bams    
run_params - individual bam run params

### single bam params
* input: bam
* group_sort_file
* log_dir
* fix_file
* sort_file
* realign_file
* intervals_file
* markdup_file
* metrics_file

In [ ]:
def define_run_params(run_info,parameters):
    # hash for storing run specific information
    from collections import defaultdict
    
    run_params = defaultdict(str)
    
#     run_info_list = run_info.split(":")
#     run_params['plat'] = run_info_list[1]
#     run_params['run_id'] = run_info_list[0]
#     run_params['vial'] = run_info_list[2]
#     run_params['rep'] = run_info_list[3]
    
#     fastq_root = parameters['root_dir'] + parameters['fastq_dir'] + \
#                                    run_info_list[1] + '/'+ 'fastq'+'/'
#     if run_info_list[1] == "MiSeq":
#         run_params['fastq1'] = fastq_root + run_info_list[0] + "_1.fastq"
#         run_params['fastq2'] = fastq_root + run_info_list[0] + "_2.fastq"
#     else:
#         run_params['fastq1'] = fastq_root + run_info_list[0] + ".fastq"
#         run_params['fastq2'] = None
        
#     run_params['out_dir'] = parameters['root_dir'] + parameters['analysis_out_dir']
#     run_params['sam'] = run_params['out_dir'] + "/tmp/" + run_params['run_id'] + ".sam"
#     run_params['bam'] = run_params['out_dir'] + "/tmp/" + run_params['run_id'] + ".bam"
#     run_params['sorted_bam'] = run_params['out_dir'] + "/"+ run_params['run_id'] + ".bam"
#     run_params['read_group'] = "@RG\tID:%s\tCN:%s\tLB:%s\tPL:%s\tSM:%s" %(run_params['run_id'],
#                                                                           parameters['center'],
#                                                                           run_params['vial']+"-"+run_params['rep'],
#                                                                           run_params['plat'],
#                                                                           run_params['run_id'])
    return run_params

In [ ]:
def dedup_realign_single_bam(run_params, bam_params):
    ''' Processing single bam file'''
    bam_group_sort(in_bam = bam_params['bam'], 
                   out_bam = bam_params['group_sort_file'], 
                   log_dir = run_params['log_dir'])
    
    bam_fixmate(in_bam = bam_params['group_sort_file'],
                out_bam = bam_params['fix_file'],
                log_dir = run_params['log_dir'])
    
    bam_sort(in_bam = bam_params['fix_file'], 
            out_bam = bam_params['sort_file'], 
            out_dir = run_params['log_dir'])
    
    bam_index(in_bam = bam_params['sort_file'],
              out_dir = run_params['log_dir'])
    
    bam_realign(in_bam = bam_params['sort_file'],
                out_bam = bam_params['realign_file'], 
                intervals_file = bam_params['intervals_file'],
                log_dir = run_params['log_dir'])
    
    bam_markdup(in_bam = bam_params['realign_file'], 
                out_bam = bam_params['markdup_file'], 
                metrics_file = bam_params['metrics_file']
                log_dir = run_params['log_dir'])
    
    bam_index(in_bam = bam_params['markdup_file'],
              out_dir = run_params['log_dir'])

In [ ]:
def run_consensus_base_pipeline(run_params,consensus_params):
    
    ## creating file with run parameters
    run_log_file = open(run_params['out_dir']+"/" + run_params['run_id'] +"-run_parameters.txt", 'w')
    run_log_file.write("Parameter\tValue\n")
    for i in run_params.keys():
        run_log_file.write("%s\t%s\n" % (i, run_params[i]))
    for i in bwa_params.keys():
        run_log_file.write("%s\t%s\n" % (i, bwa_params[i]))
    run_log_file.close()
    
    ## running bwa_mem
    bam_map_bwa_mem(ref = bwa_params['root_dir'] + bwa_params['ref'],
                        fq1 = run_params['fastq1'],
                        fq2 =run_params['fastq2'],
                        out_sam = run_params['sam'],
                        out_dir = run_params['out_dir'],
                        read_group = run_params['read_group'])
    sam_to_bam(in_sam = run_params['sam'], 
                     out_bam = run_params['bam'], 
                     out_dir = run_params['out_dir'])
    bam_sort(in_bam = run_params['bam'],
                 out_sort = run_params['sorted_bam'],
                 out_dir = run_params['out_dir'])
    bam_index(out_dir = run_params['out_dir'], bam = run_params['sorted_bam'])
    
    
    
    ## running consensus base pipeline
    

## TODO
# add merging bams and realign around indels for merged bams
genome_calls_mpileup([bam_markdup_file],ref,vcf_file,out_dir)